In [1]:
import gymnasium as gym
env = gym.make('Humanoid-v4', render_mode="rgb_array")
# env = gym.make('Humanoid-v4')

In [2]:
num_steps = 100
env.reset()
for _ in range(num_steps):
    # Choose a random action
    action = env.action_space.sample()
    
    # Perform the action and observe the next state
    observation, reward, truncated, info, done = env.step(action)
    
    # Render the current frame
    # env.render()

    # Check if the episode is done
    if done:
        # Reset the environment if the episode is done
        observation = env.reset()

# Close the environment


In [3]:
env.render()

/home/danielxu/miniconda3/envs/stepsync/lib/python3.12/site-packages/glfw/__init__.py:914: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/home/danielxu/miniconda3/envs/stepsync/lib/python3.12/site-packages/glfw/__init__.py:914: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


FatalError: an OpenGL platform library has not been loaded into this process, this most likely means that a valid OpenGL context has not been created before mjr_makeContext was called

In [4]:
env.close()